# L11: How to crunch data from Elasticsearch with python

- Elasticsearch docs: https://www.elastic.co/guide/en/elasticsearch/reference/current/index.html
- Python Elasticsearch docs: https://elasticsearch-py.readthedocs.io/en/master/

In [21]:
import pandas as pd
from elasticsearch import Elasticsearch

pd.set_option('display.max_columns', 150)

## Create the client and connect it to the cluster

In [22]:
host = 'http://104.198.35.86:9200'

es = Elasticsearch(host)

es.info()

{'name': '8d6c4856853c',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'TtO4Ay54QEaJyMnLlFIlqA',
 'version': {'number': '7.0.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'e4efcb5',
  'build_date': '2019-04-29T12:56:03.145736Z',
  'build_snapshot': False,
  'lucene_version': '8.0.0',
  'minimum_wire_compatibility_version': '6.7.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [92]:
index_0 = 'kibana_sample_data_flights'
index_1 = 'vacancies'

## Match query example ([doc](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query.html))

In [129]:
# All flights from Tokyo Haneda

query = {
    'query': {
        'match': {
            'Origin': 'Tokyo Haneda International Airport'
        }
    }
}

In [103]:
res = es.search(index=index_0, body=query)  # search in the 'Flights' index

sources = [hit['_source'] for hit in res['hits']['hits']]  # fetch only the body of each search hit

In [104]:
df = pd.DataFrame(sources).set_index('FlightNum')

df.head()

,DestCountry,OriginWeather,OriginCityName,AvgTicketPrice,DistanceMiles,FlightDelay,DestWeather,Dest,FlightDelayType,OriginCountry,dayOfWeek,DistanceKilometers,timestamp,DestLocation,DestAirportID,Carrier,Cancelled,FlightTimeMin,Origin,OriginLocation,DestRegion,OriginAirportID,OriginRegion,DestCityName,FlightTimeHour,FlightDelayMin
FlightNum,,,,,,,,,,,,,,,,,,,,,,,,,,
M21BD4I,CN,Cloudy,Tokyo,969.189931,1754.412146,True,Clear,Xi'an Xianyang International Airport,NAS Delay,JP,0,2823.452661,2019-11-04T20:17:26,"{'lat': '34.447102', 'lon': '108.751999'}",XIY,JetBeats,False,261.172633,Tokyo Haneda International Airport,"{'lat': '35.552299', 'lon': '139.779999'}",SE-BD,HND,SE-BD,Xi'an,4.352877,120
KL2UKVD,CH,Cloudy,Tokyo,347.372958,5971.560559,False,Cloudy,Zurich Airport,No Delay,JP,0,9610.295156,2019-11-04T22:34:35,"{'lat': '47.464699', 'lon': '8.54917'}",ZRH,Logstash Airways,False,600.643447,Tokyo Haneda International Airport,"{'lat': '35.552299', 'lon': '139.779999'}",CH-ZH,HND,SE-BD,Zurich,10.010724,0
IR1FR9Z,JP,Sunny,Tokyo,227.668217,0.000000,True,Clear,Tokyo Haneda International Airport,Carrier Delay,JP,0,0.000000,2019-11-04T14:16:01,"{'lat': '35.552299', 'lon': '139.779999'}",HND,JetBeats,False,270.000000,Tokyo Haneda International Airport,"{'lat': '35.552299', 'lon': '139.779999'}",SE-BD,HND,SE-BD,Tokyo,4.500000,270
J0I2OLF,PL,Clear,Tokyo,894.899226,5358.819441,False,Rain,Warsaw Chopin Airport,No Delay,JP,0,8624.183915,2019-11-04T12:43:08,"{'lat': '52.16569901', 'lon': '20.96710014'}",WAW,Logstash Airways,False,479.121329,Tokyo Haneda International Airport,"{'lat': '35.552299', 'lon': '139.779999'}",PL-MZ,HND,SE-BD,Warsaw,7.985355,0
OP4CO7M,AT,Clear,Tokyo,505.499016,5694.907068,True,Thunder & Lightning,Vienna International Airport,Carrier Delay,JP,0,9165.064521,2019-11-04T07:31:56,"{'lat': '48.11029816', 'lon': '16.56970024'}",VIE,Logstash Airways,False,641.004301,Tokyo Haneda International Airport,"{'lat': '35.552299', 'lon': '139.779999'}",AT-9,HND,SE-BD,Vienna,10.683405,30


## Query string query example ([doc](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-query-string-query.html))

In [105]:
# All the vacancies in Minsk or Kiev and with the word "Программист" in the name

query = {
    'query': {
        'query_string': {
            'query': 'address.city:(минск OR киев) AND name:(программист)'
        }
    }
}

In [106]:
res = es.search(index=index_1, body=query)  # search in the 'Vacancies' index

sources = [hit['_source'] for hit in res['hits']['hits']]

In [109]:
df = pd.DataFrame(sources).set_index('id')

df.head()

,premium,name,department,has_test,response_letter_required,area,salary,type,address,response_url,sort_point_distance,employer,published_at,created_at,archived,apply_alternate_url,insider_interview,url,alternate_url,relations,snippet,contacts,timestamp_ms
id,,,,,,,,,,,,,,,,,,,,,,,
34094600,False,Программист 1С,None,False,False,"{'id': '115', 'name': 'Киев', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}","{'city': 'Киев', 'street': 'бульвар Академика ...",None,None,"{'id': '33836', 'name': 'ВІДЖИ ТРЕЙД', 'url': ...",2019-10-15T12:52:33+0300,2019-10-15T12:52:33+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,None,https://api.hh.ru/vacancies/34094600?host=hh.ru,https://hh.ru/vacancy/34094600,[],{'requirement': 'Высшее образование. Опыт рабо...,None,1571133153000
34094600,False,Программист 1С,None,False,False,"{'id': '115', 'name': 'Киев', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}","{'city': 'Киев', 'street': 'бульвар Академика ...",None,None,"{'id': '33836', 'name': 'ВІДЖИ ТРЕЙД', 'url': ...",2019-10-15T12:52:33+0300,2019-10-15T12:52:33+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,None,https://api.hh.ru/vacancies/34094600?host=hh.ru,https://hh.ru/vacancy/34094600,[],{'requirement': 'Высшее образование. Опыт рабо...,None,1571133153000
34422421,False,Инженер-программист,None,False,False,"{'id': '115', 'name': 'Киев', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}","{'city': 'Киев', 'street': 'Бориспольская улиц...",None,None,"{'id': '7045', 'name': 'Оптима-фарм, ЛТД', 'ur...",2019-11-05T17:19:45+0300,2019-11-05T17:19:45+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,None,https://api.hh.ru/vacancies/34422421?host=hh.ru,https://hh.ru/vacancy/34422421,[],{'requirement': 'Образование - полное высшее (...,"{'name': 'Нестерова Вита', 'email': 'nesterova...",1572963585000
34411360,False,Программист РНР,None,False,False,"{'id': '115', 'name': 'Киев', 'url': 'https://...","{'from': None, 'to': 20000, 'currency': 'UAH',...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Киев', 'street': 'улица Ушинского', ...",None,None,"{'id': '3720430', 'name': 'СЕТРА', 'url': 'htt...",2019-11-05T12:10:33+0300,2019-11-05T12:10:33+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,None,https://api.hh.ru/vacancies/34411360?host=hh.ru,https://hh.ru/vacancy/34411360,[],{'requirement': 'Желаемый опыт работы PHP прог...,None,1572945033000
34422421,False,Инженер-программист,None,False,False,"{'id': '115', 'name': 'Киев', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}","{'city': 'Киев', 'street': 'Бориспольская улиц...",None,None,"{'id': '7045', 'name': 'Оптима-фарм, ЛТД', 'ur...",2019-11-05T17:19:45+0300,2019-11-05T17:19:45+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,None,https://api.hh.ru/vacancies/34422421?host=hh.ru,https://hh.ru/vacancy/34422421,[],{'requirement': 'Образование - полное высшее (...,"{'name': 'Нестерова Вита', 'email': 'nesterova...",1572963585000


## Range query example ([doc](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-range-query.html))

In [110]:
# All flights between 10 days ago from now on up to November, 13

query = {
    'query': {
        'range': {
            'timestamp': {
                # greater/less than (also "gte"/"lte" - greater/less than or equal available)
                # can be defined as ISO8601 timestamp or simply "now-NU", 
                # where N is an integer and U is the unit, e.g. "d" for days
                'gt': 'now-10d',
                'lte': '2019-11-13T06:00:00Z'
            }
        }
    }
}

In [111]:
res = es.search(index=index_0, body=query)  # search in the 'Flights' index

sources = [hit['_source'] for hit in res['hits']['hits']]

In [112]:
df = pd.DataFrame(sources).set_index('FlightNum')

df.head()

,DestCountry,OriginWeather,OriginCityName,AvgTicketPrice,DistanceMiles,FlightDelay,DestWeather,Dest,FlightDelayType,OriginCountry,dayOfWeek,DistanceKilometers,timestamp,DestLocation,DestAirportID,Carrier,Cancelled,FlightTimeMin,Origin,OriginLocation,DestRegion,OriginAirportID,OriginRegion,DestCityName,FlightTimeHour,FlightDelayMin
FlightNum,,,,,,,,,,,,,,,,,,,,,,,,,,
X98CCZO,IT,Clear,Cape Town,882.982662,5482.606665,False,Sunny,Venice Marco Polo Airport,No Delay,ZA,0,8823.400140,2019-11-04T18:27:00,"{'lat': '45.505299', 'lon': '12.3519'}",VE05,Logstash Airways,False,464.389481,Cape Town International Airport,"{'lat': '-33.96480179', 'lon': '18.60169983'}",IT-34,CPT,SE-BD,Venice,7.739825,0
UFK2WIZ,IT,Rain,Venice,190.636904,0.000000,False,Cloudy,Venice Marco Polo Airport,No Delay,IT,0,0.000000,2019-11-04T17:11:14,"{'lat': '45.505299', 'lon': '12.3519'}",VE05,Logstash Airways,False,0.000000,Venice Marco Polo Airport,"{'lat': '45.505299', 'lon': '12.3519'}",IT-34,VE05,IT-34,Venice,0.000000,0
EAYQW69,IT,Thunder & Lightning,Naples,181.694216,345.319439,True,Clear,Treviso-Sant'Angelo Airport,Weather Delay,IT,0,555.737767,2019-11-04T10:33:28,"{'lat': '45.648399', 'lon': '12.1944'}",TV01,Kibana Airlines,True,222.749059,Naples International Airport,"{'lat': '40.886002', 'lon': '14.2908'}",IT-34,NA01,IT-72,Treviso,3.712484,180
EVARI8I,CH,Clear,Zurich,180.246816,0.000000,True,Hail,Zurich Airport,Security Delay,CH,0,0.000000,2019-11-04T13:49:53,"{'lat': '47.464699', 'lon': '8.54917'}",ZRH,JetBeats,False,300.000000,Zurich Airport,"{'lat': '47.464699', 'lon': '8.54917'}",CH-ZH,ZRH,CH-ZH,Zurich,5.000000,300
M05KE88,IN,Heavy Fog,Milan,960.869736,4377.166777,True,Cloudy,Rajiv Gandhi International Airport,NAS Delay,IT,0,7044.367089,2019-11-04T12:09:35,"{'lat': '17.23131752', 'lon': '78.42985535'}",HYD,Kibana Airlines,True,602.030591,Milano Linate Airport,"{'lat': '45.445099', 'lon': '9.27674'}",SE-BD,MI11,IT-25,Hyderabad,10.033843,15


## Bool query example ([doc](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-bool-query.html))

In [122]:
# Search for all the flight sthat match several criteria: 
# the origin city is Ottawa, the flight was longer than 10 hours and it was flying within the last 15 days from now on

query = {
    'query': {
        'bool': {
            'must': [
                {
                    'match': {
                        'OriginCityName': 'Ottawa'
                    }
                },
                {
                    'query_string': {
                        'query': 'FlightTimeHour:>=10'
                    }
                },
                {
                    'range': {
                        'timestamp': {
                            'gt': 'now-15d',
                            'lte': 'now'
                        }
                    }
                }
            ]
        }
    }
}

In [123]:
res = es.search(index=index_0, body=query)

sources = [hit['_source'] for hit in res['hits']['hits']]

In [124]:
df = pd.DataFrame(sources).set_index('FlightNum')

df.head()

,DestCountry,OriginWeather,OriginCityName,AvgTicketPrice,DistanceMiles,FlightDelay,DestWeather,Dest,FlightDelayType,OriginCountry,dayOfWeek,DistanceKilometers,timestamp,DestLocation,DestAirportID,Carrier,Cancelled,FlightTimeMin,Origin,OriginLocation,DestRegion,OriginAirportID,OriginRegion,DestCityName,FlightTimeHour,FlightDelayMin
FlightNum,,,,,,,,,,,,,,,,,,,,,,,,,,
NRE9J26,AT,Clear,Ottawa,374.899128,4106.857574,False,Rain,Vienna International Airport,No Delay,CA,0,6609.346596,2019-11-04T06:23:10,"{'lat': '48.11029816', 'lon': '16.56970024'}",VIE,Logstash Airways,False,388.785094,Ottawa Macdonald-Cartier International Airport,"{'lat': '45.32249832', 'lon': '-75.66919708'}",AT-9,YOW,CA-ON,Vienna,6.479752,0
7ZTKP49,IT,Rain,Ottawa,893.927742,4036.535456,False,Cloudy,Pisa International Airport,No Delay,CA,0,6496.174118,2019-11-04T13:10:09,"{'lat': '43.683899', 'lon': '10.3927'}",PI05,ES-Air,False,309.341625,Ottawa Macdonald-Cartier International Airport,"{'lat': '45.32249832', 'lon': '-75.66919708'}",IT-52,YOW,CA-ON,Pisa,5.155694,0
5SIPOMV,CN,Heavy Fog,Ottawa,576.047399,7057.750977,False,Cloudy,Shanghai Hongqiao International Airport,No Delay,CA,0,11358.349188,2019-11-04T15:56:09,"{'lat': '31.19790077', 'lon': '121.3359985'}",SHA,Kibana Airlines,False,811.310656,Ottawa Macdonald-Cartier International Airport,"{'lat': '45.32249832', 'lon': '-75.66919708'}",SE-BD,YOW,CA-ON,Shanghai,13.521844,0
1X3WKIL,IT,Sunny,Ottawa,916.421259,3865.611623,False,Heavy Fog,Turin Airport,No Delay,CA,1,6221.098873,2019-11-05T02:40:32,"{'lat': '45.200802', 'lon': '7.64963'}",TO11,Kibana Airlines,False,327.426256,Ottawa Macdonald-Cartier International Airport,"{'lat': '45.32249832', 'lon': '-75.66919708'}",IT-21,YOW,CA-ON,Torino,5.457104,0
UYHN6BP,US,Rain,Ottawa,664.041076,376.858852,True,Clear,Philadelphia International Airport,Carrier Delay,CA,1,606.495532,2019-11-05T14:04:19,"{'lat': '39.87189865', 'lon': '-75.2410965'}",PHL,ES-Air,False,136.920817,Ottawa Macdonald-Cartier International Airport,"{'lat': '45.32249832', 'lon': '-75.66919708'}",US-PA,YOW,CA-ON,Philadelphia,2.282014,105


## Date Histogram aggregation example ([doc](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations-bucket-datehistogram-aggregation.html))

In [125]:
# How many fligths per day where delayed by more than 2 hours

query = {
    'query': {
        'query_string': {
            'query': 'FlightDelayMin:>120'
        }
    },
    'aggs': {
        'per_day': {
            'date_histogram': {
                'field': 'timestamp',
                'interval': '1d'
            }
        }
    },
    'size': 0  # 'size' is 0 in this case because we don't really need the results themselves - only the time aggregation
}

In [126]:
res = es.search(index=index_0, body=query)

buckets = res['aggregations']['per_day']['buckets']

In [128]:
df = pd.DataFrame(buckets).set_index('key_as_string')

df.head(15)

,key,doc_count
key_as_string,,
2019-11-04T00:00:00.000Z,1572825600000,48
2019-11-05T00:00:00.000Z,1572912000000,47
2019-11-06T00:00:00.000Z,1572998400000,62
2019-11-07T00:00:00.000Z,1573084800000,61
2019-11-08T00:00:00.000Z,1573171200000,57
2019-11-09T00:00:00.000Z,1573257600000,50
2019-11-10T00:00:00.000Z,1573344000000,41
2019-11-11T00:00:00.000Z,1573430400000,63
2019-11-12T00:00:00.000Z,1573516800000,60
